In [1]:
import psi4
import numpy as np
import sys
sys.path.append('../lib')
from P4toC4_aux import *

In [9]:
#  C4: STO-3G  -74.963 319 056 386 7

psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "6-31G*")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

# O
# H 1 0.96
# H 1 0.96 2 104.5


h2o = psi4.geometry("""
0 1
O            0.000000000000     0.000000000000    -0.065775570547
H            0.000000000000    -0.759061990794     0.521953018286
H            0.000000000000     0.759061990794     0.521953018286
symmetry c1
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-76.00900934211914

In [10]:
basisset=wf.basisset()
mol=wf.molecule()
for atom in range(mol.natom()):
    print(mol.symbol(atom))
basisset.nbf()

O
H
H


18

In [11]:
# stolen from molden writer v1.5 and modified
mol_string = '\n[GTO]\n'
for atom in range(mol.natom()):
    mol_string += f"  {atom+1:d} 0\n"
    #i = 0
    for rel_shell_idx in range(basisset.nshell_on_center(atom)):
        #abs_shell_idx = basisset.shell_on_center(atom, rel_shell_idx)
        #shell = basisset.shell(abs_shell_idx)
        shell = basisset.shell(atom, rel_shell_idx)
        mol_string += f" {shell.amchar:s}{shell.nprimitive:5d}  1.00\n"
        for prim in range(shell.nprimitive):
            mol_string += f"{shell.exp(prim):20.10f} {shell.original_coef(prim):20.10f}\n"
    mol_string += '\n'

print(mol_string)


[GTO]
  1 0
 s    6  1.00
     5484.6717000000         0.0018311000
      825.2349500000         0.0139501000
      188.0469600000         0.0684451000
       52.9645000000         0.2327143000
       16.8975700000         0.4701930000
        5.7996353000         0.3585209000
 s    3  1.00
       15.5396160000        -0.1107775000
        3.5999336000        -0.1480263000
        1.0137618000         1.1307670000
 p    3  1.00
       15.5396160000         0.0708743000
        3.5999336000         0.3397528000
        1.0137618000         0.7271586000
 s    1  1.00
        0.2700058000         1.0000000000
 p    1  1.00
        0.2700058000         1.0000000000
 d    1  1.00
        0.8000000000         1.0000000000

  2 0
 s    3  1.00
       18.7311370000         0.0334946000
        2.8253937000         0.2347269500
        0.6401217000         0.8137573300
 s    1  1.00
        0.1612778000         1.0000000000

  3 0
 s    3  1.00
       18.7311370000         0.0334946000
       

In [12]:
ao=0
for atom in range(mol.natom()):
    print(f"Atom  {atom}")
    for rel_shell_idx in range(basisset.nshell_on_center(atom)):
        #abs_shell_idx = basisset.shell_on_center(atom, rel_shell_idx)
        #shell = basisset.shell(abs_shell_idx)
        shell = basisset.shell(atom, rel_shell_idx)
        print(f" {rel_shell_idx}  {shell.amchar}")

Atom  0
 0  s
 1  s
 2  p
 3  s
 4  p
 5  d
Atom  1
 0  s
 1  s
Atom  2
 0  s
 1  s


In [6]:
for ishell in range(basisset.nshell()):
    shell = basisset.shell(ishell)
    print(f"Shell {ishell}  {shell.amchar} {shell.am}")

Shell 0  s 0
Shell 1  s 0
Shell 2  p 1
Shell 3  s 0
Shell 4  s 0


In [7]:
offset = ao_offset(basisset, verbose=0)
offset

array([ 0,  0,  2, -1, -1,  0,  0])

In [1]:
# from write_nbo:
pure_order = [
    [1],  # s
    [103, 101, 102],  # p
    [255, 252, 253, 254, 251],  # d: z2 xz yz x2-y2 xy
    [351, 352, 353, 354, 355, 356, 357],  # f
    [451, 452, 453, 454, 455, 456, 457, 458, 459],  #g
    [551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561]  #h
]

# from write molden
    '''
    Reordering expected by Molden
    P: x, y, z
    5D: D 0, D+1, D-1, D+2, D-2
    6D: xx, yy, zz, xy, xz, yz
    7F: F 0, F+1, F-1, F+2, F-2, F+3, F-3
    10F: xxx, yyy, zzz, xyy, xxy, xxz, xzz, yzz, yyz, xyz
    '''
molden_cartesian_order = [
    [2,0,1,0,0,0,0,0,0,0,0,0,0,0,0], # p
    [0,3,4,1,5,2,0,0,0,0,0,0,0,0,0], # d
    [0,4,5,3,9,6,1,8,7,2,0,0,0,0,0], # f
    [0,3,4,9,12,10,5,13,14,7,1,6,11,8,2] # g
]